
---

# 📊 Proyecto Final: Inteligencia de Negocios y Minería de Datos

## 📘 Informe de Proyecto Final

**Materia:** Inteligencia de Negocios (ICC-321-T)
**Tema:** Dashboard Interactivo y Modelo de Minería de Datos Descriptivo

**Autores:**

* Randy Alexander Germosén Ureña *(1013-4707)*
* Fernando Almonte Delgado *(1015-7628)*

**Repositorio:**
[icc321-2025-final](https://github.com/TZeik/icc321-2025-final) <img src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" width="15" height="15"/>

---

## 🎯 Objetivo del Proyecto

El propósito de este proyecto es desarrollar una solución integral de Inteligencia de Negocios utilizando datos públicos del gobierno de la República Dominicana. Consta de dos componentes principales:

1. **Dashboard Interactivo:** Permite visualizar y monitorear métricas de gasto y nómina para apoyar la toma de decisiones.
2. **Modelo de Minería de Datos:** Implementación de un modelo descriptivo (Clustering) para descubrir patrones y segmentar perfiles de empleados.

---

## 📂 Datasets Utilizados

Se procesaron y unificaron datos históricos abarcando el periodo **2018–2025**:

1. **Nómina de la Contraloría General de la República:**
   Información detallada sobre empleados, cargos, departamentos y sueldos.
2. **Índice de Precios al Consumidor (IPC):**
   Datos del Banco Central utilizados para calcular el **salario real** (ajustado por inflación) en comparación con el salario nominal.

---

## 🧠 Metodología

El desarrollo del proyecto se estructuró en las siguientes fases técnicas:

### 1. Ingeniería de Datos (ETL)

* **Extracción y Limpieza:**

  * Unificación de múltiples archivos CSV mensuales/anuales.
  * Estandarización de nombres de cargos, normalización de formatos monetarios y corrección de codificación (`latin-1`, `utf-8`).
  * Homogeneización de los nombres de los meses.
* **Enriquecimiento:**

  * Cruce entre nómina e IPC para calcular la pérdida de poder adquisitivo.

### 2. Almacenamiento (Data Warehousing)

* Implementación de un **Data Warehouse** local con **SQLite**.
* Diseño bajo un **Esquema en Estrella**, con:

  * Tabla de hechos: `fact_nomina`
  * Tablas de dimensiones: `dim_empleado`, `dim_tiempo`

### 3. Visualización (Dashboard)

* Creación de Dashboard interactivo en **Tableau Public**.
* Diseño de KPIs como:

  * Gasto total,
  * Brecha salarial,
  * Evolución de plantilla,
  * Tendencias del salario real vs nominal.

### 4. Minería de Datos (Machine Learning)

* **Preprocesamiento:**
  Codificación de variables categóricas y escalado numérico.
* **Modelado:**
  Aplicación de **K-Means Clustering** para identificar grupos de empleados con características similares.
* **Evaluación:**

  * Método del Codo
  * Coeficiente de Silueta

---

## 📊 Resultados Principales

La solución permite analizar hallazgos relevantes como:

* Diferencias entre **Sueldo Nominal** y **Sueldo Real** a lo largo del tiempo.
* Identificación de departamentos con mayor incremento en el gasto de nómina.
* Clusters de empleados basados en sueldo, cargo y antigüedad, revelando patrones ocultos en la organización.

---

## 🧩 Herramientas Utilizadas

### Lenguajes y Entorno

* **Python 3.x** (Jupyter Notebook)

### Librerías Principales

* `pandas` — Manipulación y limpieza de datos
* `sqlite3` — Data Warehouse local
* `scikit-learn` — Algoritmo K-Means y métricas
* `matplotlib` — Visualización del método del codo

### Visualización

* **Tableau Public** — Dashboard interactivo final

---



In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
# Diccionario para estandarizar meses a números
meses_map = {
    'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
    'JULIO': 7, 'AGOSTO': 8, 'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
}

def limpiar_moneda(valor):
    """Convierte '100,000.00' o 80000 a float puro"""
    if pd.isna(valor): return 0.0
    # Convertir a string, quitar 'RD$', comas y espacios
    s = str(valor).replace('RD$', '').replace(',', '').strip()
    try:
        return float(s)
    except:
        return 0.0

def limpiar_texto(texto):
    """Quita espacios extra y pone todo en mayúsculas"""
    if pd.isna(texto): return "DESCONOCIDO"
    return str(texto).strip().upper()

print("Funciones configuradas.")

Funciones configuradas.


In [3]:
# Cargar IPC
df_ipc = pd.read_csv('./raw_data/ipc_base_1984-2025.csv', sep=';')

# Limpieza IPC
# Filtramos solo desde 2018 en adelante para coincidir con la nómina
df_ipc = df_ipc[df_ipc['PERIODO'] >= 2018].copy()

# Estandarizamos mes para poder cruzar
df_ipc['MES_LIMPIO'] = df_ipc['MES'].apply(limpiar_texto).map(meses_map)

# Seleccionamos solo lo necesario: Año, Mes numérico y el Índice
df_ipc_clean = df_ipc[['PERIODO', 'MES_LIMPIO', 'INDICE']].rename(columns={
    'PERIODO': 'ANIO',
    'MES_LIMPIO': 'MES_NUM',
    'INDICE': 'IPC'
})

print(f"IPC procesado: {len(df_ipc_clean)} registros (2018-2025)")
# Vista previa rápida
display(df_ipc_clean.head(10))

IPC procesado: 94 registros (2018-2025)


,ANIO,MES_NUM,IPC
408,2018,1,94.58
409,2018,2,94.48
410,2018,3,94.83
411,2018,4,95.21
412,2018,5,95.46
413,2018,6,95.66
414,2018,7,95.64
415,2018,8,95.67
416,2018,9,95.74
417,2018,10,95.96


In [4]:
# Cargar Nómina
df_nomina = pd.read_csv(
    './raw_data/nomina-empleados-fijos-y-contratados-CSV-2018-2025.csv',
    sep=';',
    encoding='latin-1',
    low_memory=False
)

# Normalizar nombres de columnas
df_nomina.columns = (
    df_nomina.columns
    .str.strip()
    .str.upper()
    .str.normalize("NFKD")
    .str.encode("ascii", errors="ignore")
    .str.decode("utf-8")
)

# Renombrar AO → ANIO (si aparece así)
df_nomina.columns = [c.replace('AO', 'ANIO') for c in df_nomina.columns]

# LIMPIAR SUELDO
df_nomina['SUELDO_LIMPIO'] = df_nomina['SUELDO'].apply(limpiar_moneda)

# LIMPIAR TEXTOS
cols_texto = ['NOMBRE', 'FUNCION', 'DEPARTAMENTO', 'ESTATUS']
for col in cols_texto:
    if col in df_nomina.columns:
        df_nomina[col] = df_nomina[col].astype(str).apply(limpiar_texto)

# LIMPIAR Y EXTRAER AÑO (solución al error)
df_nomina['ANIO'] = (
    df_nomina['ANIO']
    .astype(str)
    .str.extract(r'(\d{4})') # Extrae el primer año válido
)

df_nomina['ANIO'] = pd.to_numeric(df_nomina['ANIO'], errors='coerce')

# Correcciones latin-1 vs. utf-8
df_nomina['NOMBRE'] = df_nomina['NOMBRE'].str.replace('Ï¿½', 'Ñ', regex=False)
df_nomina['FUNCION'] = df_nomina['FUNCION'].str.replace('INGENIERÏ¿½A', 'INGENIERIA', regex=False)
df_nomina['FUNCION'] = df_nomina['FUNCION'].str.replace('DISEÏ¿½ADOR', 'DISEÑADOR', regex=False)
df_nomina['DEPARTAMENTO'] = df_nomina['DEPARTAMENTO'].str.replace('AUDITORÏ¿½A', 'AUDITORIA', regex=False)
df_nomina['DEPARTAMENTO'] = df_nomina['DEPARTAMENTO'].str.replace('DIRECCIÏ¿½N', 'DIRECCION', regex=False)
df_nomina['DEPARTAMENTO'] = df_nomina['DEPARTAMENTO'].str.replace('DIVISIÏ¿½N', 'DIVISION', regex=False)
df_nomina['DEPARTAMENTO'] = df_nomina['DEPARTAMENTO'].str.replace('CAPACITACIÏ¿½N', 'CAPACITACION', regex=False)

# Estandarizar Mes a Número
df_nomina['MES_NUM'] = (
    df_nomina['MES']
    .astype(str)
    .apply(limpiar_texto)
    .map(meses_map)
)

# Eliminar filas inválidas (mes o año no legibles)
df_nomina.dropna(subset=['ANIO', 'MES_NUM'], inplace=True)

df_nomina['ANIO'] = df_nomina['ANIO'].astype(int)
df_nomina['MES_NUM'] = df_nomina['MES_NUM'].astype(int)

# RESULTADO
print(f"Nómina procesada: {len(df_nomina)} empleados.")
display(df_nomina.head(5))


Nómina procesada: 152728 empleados.


,NOMBRE,FUNCION,DEPARTAMENTO,SUELDO,ESTATUS,MES,ANIO,SUELDO_LIMPIO,MES_NUM
0,NOEL LUPERON RAMIREZ,ASESOR ACADEMICO,CENTRO DE CAPACITACION CGR,80000,CONTRATADO,ENERO,2018,80000.0,1
1,PAOLA MARITZA POLANCO RODRIGUEZ,COORDINADOR ACADEMICO,CENTRO DE CAPACITACION CGR,40000,FIJOS,ENERO,2018,40000.0,1
2,ANA IRIS MARTINEZ NUÑEZ,DIGITADOR(A),CONSULTORIA JURIDICA,20000,CONTRATADO,ENERO,2018,20000.0,1
3,ANGEL FRANCISCO ROMAN CORCINO,AUXILIAR ADMINISTRATIVO(A),CONSULTORIA JURIDICA,25000,FIJOS,ENERO,2018,25000.0,1
4,ANNETTE ALTAGRACIA PEÑA ACOSTA,ENCARGADO DIVISION,CONSULTORIA JURIDICA,60000,FIJOS,ENERO,2018,60000.0,1


In [5]:
# Unir Nómina con IPC usando Año y Mes
df_final = pd.merge(df_nomina, df_ipc_clean, on=['ANIO', 'MES_NUM'], how='left')

# ENRIQUECIMIENTO: Calcular Salario Real
# Fórmula: (Salario Nominal / IPC Actual) * IPC Base (Usaremos 100 como base conceptual o el IPC del primer registro)
# Para simplificar, ajustaremos a valor de 2018: Salario / IPC * IPC_2018
ipc_base = df_ipc_clean['IPC'].min() # Tomamos el menor IPC del periodo como base
df_final['SUELDO_REAL'] = (df_final['SUELDO_LIMPIO'] / df_final['IPC']) * ipc_base

# CARGA A SQLITE (Data Warehouse)
conn = sqlite3.connect('DW_Nomina_Publica.db')

# Crear Tabla de Hechos (Fact Table)
# Guardamos las métricas numéricas y claves
fact_table = df_final[['ANIO', 'MES_NUM', 'SUELDO_LIMPIO', 'SUELDO_REAL', 'IPC', 'FUNCION', 'DEPARTAMENTO', 'ESTATUS', 'NOMBRE']]
fact_table.to_sql('fact_nomina', conn, if_exists='replace', index=False)

print("Data Warehouse creado: 'DW_Nomina_Publica.db'")
conn.close()

Data Warehouse creado: 'DW_Nomina_Publica.db'
